**Лабораторная работа №3** Вариант 9

**Цель лабораторной работы:** изучить программные средства, используемые для визуализации наборов данных.

Выполним анализ набора данных «Анализ текучести кадров в компании». Набор данных содержит информацию о сотрудниках компании, включая их образование, историю работы, демографию и факторы, связанные с занятостью. Необходимо проанализировать проблемы, связанные с текучестью кадров в компании. Набор данных включает следующие атрибуты:

* **satisfaction_level** – уровень удовлетворенности (0-1)
* **last_evaluation** – время, прошедшее с момента последней оценки в годах
* **number_projects** – количество выполненных проектов
* **average_monthly_hours** – среднее количество рабочих часов за месяц
* **time_spend_company** – время работы в компании в годах
* **work_accident** – у сотрудника был несчастный случай на рабочем месте
* **promotion_last_5years** – было ли у сотрудника повышение за последние 5 лет
* **sales** – отдел, в котором работает сотрудник
* **salary** – уровень заработной платы
* **left** – работник уволился или нет (1 или 0) – целевая переменная

**1. Импортируем библиотеки и загружаем данные.**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc

df = pd.read_csv("/employee.csv")
df.head()

**2. Выведем целевую переменную, целевые имена и частоту каждого уникального значения.**

In [ ]:
(unique, counts) = np.unique(df['left'], return_counts=True)
print('Unique values of the target variable: ', unique)
print('Counts of the target variable:', counts)

sns.set_style("whitegrid")
sns.countplot(data=df, x='left')

In [ ]:
df.info()

**3. Проверяем наличие пропущенных значений и выбросов.**

In [ ]:
print(df.isnull().sum())

In [ ]:
sns.boxplot(x=df['satisfaction_level'])

**4. Исключаем невлияющие категориальные признаки.**

In [ ]:
df_copy = df.copy()
non_influencing_categorical_features = ['department']
df_copy.drop(non_influencing_categorical_features, axis=1, inplace=True)
df_copy.head()

In [ ]:
salary_map = {'low': 0, 'medium': 1, 'high': 2}
df_copy['salary'] = df_copy['salary'].map(salary_map)
df_copy.head()

**5. Разделяем данные на признаки и целевую переменную.**

In [587]:
y = df_copy['left']
X = df_copy.drop('left', axis=1)

**6. Применяем операцию нормализации для численной устойчивости.**

In [588]:
standardizer = StandardScaler()
X_scaled = standardizer.fit_transform(X)

**7. Разделяем данные на тренировочный и тестовый наборы.**

In [589]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=0)

**8. Обучаем модель логистической регрессии**

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

**9. Делаем прогноз на основе данных тестирования**

In [591]:
y_pred = model.predict(X_test)

**10. Рассчитываем показатели точности, сравнив фактические и прогнозируемые значения**

In [ ]:
cm = confusion_matrix(y_test, y_pred)
TN, FP, FN, TP = confusion_matrix(y_test, y_pred).ravel()

print('True Positive(TP) = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN) = ', TN)
print('False Negative(FN) = ', FN)

accuracy = (TP+TN) / (TP+FP+TN+FN)
print('Accuracy of the binary classification = {:.3f}'.format(accuracy))

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

In [ ]:
sns.heatmap(cm, square=True, annot=True, fmt='d', cbar=False, cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность: {accuracy:.2f}")

In [ ]:
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

In [ ]:
f1 = f1_score(y_test, y_pred)
print(f"F1-мера: {f1:.2f}")

In [ ]:
df_copy.corr()

**11. Строим ROC-кривую.**

In [ ]:
y_score = model.predict_proba(X_test)[:, 1]

fpr, tpr, thresholds = roc_curve(y_test, y_score)

roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

print(f"AUC: {roc_auc:.3f}")

**12. Анализ важности признаков.**

In [ ]:
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': abs(model.coef_[0])
}).sort_values('importance', ascending=True)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance['feature'], feature_importance['importance'])
plt.title('Feature Importance')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

print("Важность признаков:")
print(feature_importance.sort_values('importance', ascending=False))

Таким образом, в лабораторной работе №3 был проведен анализ датасета employee.csv с помощью метода логистической регрессии, была получена модель, определяющая причины ухода сотрудников из компании.